In [4]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# 1. Набор отзывов ("мешок слов")
feedback = [
    "Полнейшая ерунда! Потратил время впустую.",
    "Просто шедевр, рекомендую каждому!",
    "Скучно, затянуто, герои никакие.",
    "Эмоциональный и трогательный сюжет.",
    "Неплохие эффекты, но сюжет слабый.",
    "Не понимаю, зачем это сняли.",
    "Лучший фильм, который я видел за последние годы.",
    "Слишком клишировано, ничего нового.",
    "Фильм, который заставляет задуматься.",
    "Отличное развлечение для всей семьи.",
    "Повествование слишком хаотичное.",
    "Фильм, который вдохновляет на подвиги.",
    "Совсем не оправдал ожиданий.",
    "Очень необычный и смелый сюжет.",
    "Не могу сказать, что понравилось.",
    "Настоящий подарок для ценителей жанра.",
    "Безусловно, лучший фильм года!",
    "Местами интересно, но в целом слабовато.",
    "Сюжет простоват, но актерская игра отличная.",
    "Это просто восторг! Все на высшем уровне."
]

# 2. Создаем список стоп-слов и функцию для очистки текста
custom_stop_words = set(ENGLISH_STOP_WORDS).union(
    {"это", "этот", "но", "для", "все", "фильм", "очень", "нет", "есть", "был", "как", "что"}
)

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Удаляем пунктуацию
    tokens = text.split()
    return ' '.join([word for word in tokens if word not in custom_stop_words])

# Применяем очистку текста
processed_reviews = [preprocess_text(review) for review in reviews]

# 3. TF-IDF векторизация
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_reviews)

print("TF-IDF матрица:")
print(tfidf_matrix.toarray())
print("Слова:", tfidf_vectorizer.get_feature_names_out())

# 4. Векторизация с использованием n-грамм
ngram_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
ngram_matrix = ngram_vectorizer.fit_transform(processed_reviews)

print("N-граммы матрица:")
print(ngram_matrix.toarray())
print("N-граммы:", ngram_vectorizer.get_feature_names_out())

# 5. Pipeline: Векторизатор + Регрессор
ratings = np.array([9, 2, 8, 4, 6, 3, 9, 3, 7, 8, 4, 9, 5, 8, 6, 8, 2, 9, 7, 2])  # Пример оценок

pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 3))),
    ('regressor', LinearRegression())
])

pipeline.fit(reviews, ratings)
predicted_ratings = pipeline.predict(reviews)
print("Предсказанные оценки:", predicted_ratings)

# 6. Исследование коэффициентов модели
feature_names = pipeline.named_steps['vectorizer'].get_feature_names_out()
coefficients = pipeline.named_steps['regressor'].coef_

sorted_features = sorted(zip(coefficients, feature_names), reverse=True)
print("Наиболее важные признаки:")
for coef, feat in sorted_features[:10]:
    print(f"{feat}: {coef:.3f}")



TF-IDF матрица:
[[0.45774055 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Слова: ['актеров' 'актерская' 'большего' 'бы' 'великолепная' 'впечатлило'
 'времени' 'всем' 'выбор' 'вызвал' 'глубокий' 'года' 'динамично' 'еще'
 'жалко' 'затянуто' 'захватывающе' 'захватывающий' 'игра' 'из'
 'интересные' 'интересный' 'клише' 'комедий' 'который' 'любителям'
 'местами' 'много' 'множество' 'не' 'никакого' 'нужно' 'один' 'ожидал'
 'отличная' 'отличный' 'персонажи' 'понравилось' 'понравился' 'понять'
 'посмотреть' 'потраченного' 'потрясающий' 'превосходный' 'предсказуемый'
 'просмотра' 'просто' 'пустой' 'раз' 'режиссура' 